This notebook is a rough draft and has not been cleaned or refactored

In [1]:
import pandas as pd
import numpy as np
import gensim
import os
from gensim.models.doc2vec import *

In [2]:
df = pd.read_csv(os.getcwd() + '/' + '100Paragraphs_150Words.csv', index_col = 0)

In [3]:
df.head()

,text,author,sex,period
0,than ever over at the house the main building ...,KateChopin,female,realism
1,lost in some inward maze of contemplation or t...,KateChopin,female,realism
2,about himself he was very young and did not kn...,KateChopin,female,realism
3,his wife with her inattention her habitual neg...,KateChopin,female,realism
4,on hand to say goodby to him his wife stood sm...,KateChopin,female,realism


In [4]:
authors = list(set((df['author'])))
authors

['MaryWollstonecraft',
 'VirginiaWoolf',
 'MarkTwain',
 'FScottFitzgerald',
 'JaneAustin',
 'MargaretFuller',
 'KateChopin',
 'NathanielHawthorne',
 'JohnLocke',
 'HenryDavidThoreau',
 'EdithWharton',
 'MaryShelley',
 'JackLondon',
 'CharlesDickens']

In [6]:
# split 
def split_by_authors():
    df_split ={}
    for author in authors:
        df_ = df[df['author'] == author]
        df_split[author] = df_
    return df_split

df_by_authors = split_by_authors()

In [7]:
train_set_df = pd.DataFrame(columns=list(df.columns))
test_set_df = pd.DataFrame(columns=list(df.columns))
validation_set_df = pd.DataFrame(columns=list(df.columns))
for author in authors:
    train_set = df_by_authors[author].sample(70)
    test_val_set = df_by_authors[author].drop(train_set.index)
    validation_set = test_val_set.sample(5)
    test_set = test_val_set.drop(validation_set.index)
    train_set_df = pd.concat([train_set_df,train_set])
    test_set_df = pd.concat([test_set_df,test_set])
    validation_set_df = pd.concat([validation_set_df,validation_set])

In [8]:
train_set_df = train_set_df.reset_index()
test_set_df = test_set_df.reset_index()
validation_set_df = validation_set_df.reset_index()

In [9]:
# Create TaggedDocument objects for train_corpus
train_corpus = []
for i in range(0, len(train_set_df)):
    doc = TaggedDocument(train_set_df['text'][i].split(), [train_set_df['author'][i]])   
    train_corpus.append(doc)

In [10]:
train_corpus[10]

TaggedDocument(words=['almost', 'to', 'adoration', 'truefor', 'never', 'was', 'there', 'a', 'sensualist', 'who', 'paid', 'more', 'fervent', 'adoration', 'at', 'the', 'shrine', 'of', 'beauty', 'so', 'devout', 'indeed', 'was', 'his', 'respect', 'for', 'the', 'person', 'that', 'excepting', 'the', 'virtue', 'of', 'chastity', 'for', 'obvious', 'reasons', 'he', 'only', 'wished', 'to', 'see', 'it', 'embellished', 'by', 'charms', 'weaknesses', 'and', 'errors', 'he', 'was', 'afraid', 'lest', 'the', 'austerity', 'of', 'reason', 'should', 'disturb', 'the', 'soft', 'playfulness', 'of', 'love', 'the', 'master', 'wished', 'to', 'have', 'a', 'meretricious', 'slave', 'to', 'fondle', 'entirely', 'dependent', 'on', 'his', 'reason', 'and', 'bounty', 'he', 'did', 'not', 'want', 'a', 'companion', 'whom', 'he', 'should', 'be', 'compelled', 'to', 'esteem', 'or', 'a', 'friend', 'to', 'whom', 'he', 'could', 'confide', 'the', 'care', 'of', 'his', 'childrens', 'education', 'should', 'death', 'deprive', 'them', '

In [11]:
# Create TaggedDocument objects for test_corpus
test_corpus = []
for i in range(0, len(test_set_df)):
    doc = test_set_df['text'][i].split()
    test_corpus.append(doc)

In [12]:
test_corpus[10]

['of',
 'man',
 'the',
 'homage',
 'of',
 'passion',
 'man',
 'accustomed',
 'to',
 'bow',
 'down',
 'to',
 'power',
 'in',
 'his',
 'savage',
 'state',
 'can',
 'seldom',
 'divest',
 'himself',
 'of',
 'this',
 'barbarous',
 'prejudice',
 'even',
 'when',
 'civilization',
 'determines',
 'how',
 'much',
 'superior',
 'mental',
 'is',
 'to',
 'bodily',
 'strength',
 'and',
 'his',
 'reason',
 'is',
 'clouded',
 'by',
 'these',
 'crude',
 'opinions',
 'even',
 'when',
 'he',
 'thinks',
 'of',
 'the',
 'deity',
 'his',
 'omnipotence',
 'is',
 'made',
 'to',
 'swallow',
 'up',
 'or',
 'preside',
 'over',
 'his',
 'other',
 'attributes',
 'and',
 'those',
 'mortals',
 'are',
 'supposed',
 'to',
 'limit',
 'his',
 'power',
 'irreverently',
 'who',
 'think',
 'that',
 'it',
 'must',
 'be',
 'regulated',
 'by',
 'his',
 'wisdom',
 'i',
 'disclaim',
 'that',
 'species',
 'of',
 'humility',
 'which',
 'after',
 'investigating',
 'nature',
 'stops',
 'at',
 'the',
 'author',
 'the',
 'high',
 'a

In [13]:
# Declare gyperparamters 
vec_size = 25
min_count = 2
epochs = 30
alpha = 0.030
min_alpha = 0.030

In [14]:
# Doc2Vec - initializing the model

model = Doc2Vec(vector_size=vec_size,
                min_count=min_count,
                epochs=epochs,
                alpha = alpha,
                min_alpha = min_alpha)

model.build_vocab(train_corpus)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 9.49 s, sys: 548 ms, total: 10 s
Wall time: 4.41 s


In [15]:
len(model.docvecs)
# should be 14
# two for gender, seven for periods, and fourteen for authors 

14

In [ ]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    print(doc_id)
    print(sims)
    rank = [doc_id for doc_id, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

# see how many it got correct and how many it didn't 
collections.Counter(ranks)

In [16]:
train_corpus[600]

TaggedDocument(words=['it', 'should', 'be', 'thought', 'to', 'extend', 'to', 'life', 'and', 'death', 'at', 'any', 'time', 'over', 'their', 'children', 'more', 'than', 'over', 'any', 'body', 'else', 'neither', 'can', 'there', 'be', 'any', 'pretence', 'why', 'this', 'parental', 'power', 'should', 'keep', 'the', 'child', 'when', 'grown', 'to', 'a', 'man', 'in', 'subjection', 'to', 'the', 'will', 'of', 'his', 'parents', 'any', 'farther', 'than', 'having', 'received', 'life', 'and', 'education', 'from', 'his', 'parents', 'obliges', 'him', 'to', 'respect', 'honour', 'gratitude', 'assistance', 'and', 'support', 'all', 'his', 'life', 'to', 'both', 'father', 'and', 'mother', 'and', 'thus', 'tis', 'true', 'the', 'paternal', 'is', 'a', 'natural', 'government', 'but', 'not', 'at', 'all', 'extending', 'itself', 'to', 'the', 'ends', 'and', 'jurisdictions', 'of', 'that', 'which', 'is', 'political', 'the', 'power', 'of', 'the', 'father', 'doth', 'not', 'reach', 'at', 'all', 'to', 'the', 'property', 'o

In [17]:
inferred_vector = model.infer_vector(train_corpus[600].words)
sims = model.docvecs.most_similar([inferred_vector])
# print(inferred_vector)
sims

[('JohnLocke', 0.6411967873573303),
 ('MaryWollstonecraft', 0.562842845916748),
 ('MargaretFuller', 0.3735983967781067),
 ('NathanielHawthorne', 0.3670194745063782),
 ('JackLondon', 0.33635133504867554),
 ('MarkTwain', 0.3362114727497101),
 ('JaneAustin', 0.32208365201950073),
 ('CharlesDickens', 0.29809311032295227),
 ('VirginiaWoolf', 0.25395870208740234),
 ('FScottFitzgerald', 0.24994030594825745)]

In [18]:
test_set_df.loc[20]

index                                                   683
text      but the animal spirits the individual characte...
author                                   MaryWollstonecraft
sex                                                  female
period                                        enlightenment
Name: 20, dtype: object

In [19]:
inferred_vector = model.infer_vector(test_corpus[20])
sims = model.docvecs.most_similar([inferred_vector])
# print(inferred_vector)
sims[1]
sims

[('MaryWollstonecraft', 0.68779057264328),
 ('MargaretFuller', 0.4038630723953247),
 ('NathanielHawthorne', 0.3386878967285156),
 ('HenryDavidThoreau', 0.3342222571372986),
 ('MaryShelley', 0.31416055560112),
 ('VirginiaWoolf', 0.24206586182117462),
 ('JaneAustin', 0.21889840066432953),
 ('EdithWharton', 0.21500596404075623),
 ('JackLondon', 0.1918397843837738),
 ('MarkTwain', 0.18339398503303528)]

In [23]:
test_set_df.loc[40]

index                                                   765
text      whose suspicions were not yet laid to rest ive...
author                                        VirginiaWoolf
sex                                                  female
period                                      early_modernism
Name: 40, dtype: object

In [24]:
inferred_vector = model.infer_vector(test_corpus[40])
sims = model.docvecs.most_similar([inferred_vector])
# print(inferred_vector)
sims[1]
sims

[('VirginiaWoolf', 0.7001662850379944),
 ('HenryDavidThoreau', 0.3671192526817322),
 ('MargaretFuller', 0.35480695962905884),
 ('JackLondon', 0.33217301964759827),
 ('NathanielHawthorne', 0.329555481672287),
 ('FScottFitzgerald', 0.31974852085113525),
 ('EdithWharton', 0.28611868619918823),
 ('JaneAustin', 0.27271559834480286),
 ('MarkTwain', 0.2503357529640198),
 ('KateChopin', 0.24139408767223358)]

In [ ]:
print(sims[1][1])
print(type(sims[1][1]))

In [22]:
model.docvecs.doctags

{'CharlesDickens': Doctag(offset=13, word_count=10570, doc_count=70),
 'EdithWharton': Doctag(offset=10, word_count=10570, doc_count=70),
 'FScottFitzgerald': Doctag(offset=3, word_count=10570, doc_count=70),
 'HenryDavidThoreau': Doctag(offset=9, word_count=10570, doc_count=70),
 'JackLondon': Doctag(offset=12, word_count=10570, doc_count=70),
 'JaneAustin': Doctag(offset=4, word_count=10570, doc_count=70),
 'JohnLocke': Doctag(offset=8, word_count=10570, doc_count=70),
 'KateChopin': Doctag(offset=6, word_count=10570, doc_count=70),
 'MargaretFuller': Doctag(offset=5, word_count=10570, doc_count=70),
 'MarkTwain': Doctag(offset=2, word_count=10570, doc_count=70),
 'MaryShelley': Doctag(offset=11, word_count=10570, doc_count=70),
 'MaryWollstonecraft': Doctag(offset=0, word_count=10570, doc_count=70),
 'NathanielHawthorne': Doctag(offset=7, word_count=10570, doc_count=70),
 'VirginiaWoolf': Doctag(offset=1, word_count=10570, doc_count=70)}

In [ ]:
model.docvecs.index2entity[0]

In [ ]:
model.docvecs.most_similar_to_given
model.docvecs.n_similarity
model.docvecs.rank
model.docvecs.doctag_syn0norm

Trying out some of the analogical reasoning

THIS WAS FOR OLD MODEL

In [20]:
locke_male_to_female = model.docvecs[0] - model.docvecs[2] + model.docvecs[9]
sims = model.docvecs.most_similar([locke_male_to_female])
sims

[('JohnLocke', 0.7043590545654297),
 ('JaneAustin', 0.5777753591537476),
 ('female', 0.4133979082107544),
 ('MaryShelley', 0.3785366714000702),
 ('KateChopin', 0.276517391204834),
 ('enlightenment', 0.2373473048210144),
 ('victorian', 0.22287753224372864),
 ('HenryDavidThoreau', 0.08393082022666931),
 ('gothic/romantic', 0.08032718300819397),
 ('realism', 0.05206068605184555)]

In [21]:
twain_male_to_female = np.add(np.subtract(model.docvecs[17], model.docvecs[2]), model.docvecs[9])
sims = model.docvecs.most_similar([twain_male_to_female])
sims

[('MarkTwain', 0.7890018224716187),
 ('female', 0.32558053731918335),
 ('realism', 0.3135584592819214),
 ('MaryWollstonecraft', 0.27268674969673157),
 ('MaryShelley', 0.1598474681377411),
 ('MargaretFuller', 0.14321206510066986),
 ('EdithWharton', 0.133916974067688),
 ('VirginiaWoolf', 0.06805793195962906),
 ('JaneAustin', 0.06686685979366302),
 ('gothic/romantic', 0.030892416834831238)]

In [ ]:
twain_real_to_vict = model.docvecs[0] - model.docvecs[1] + model.docvecs[6] 
sims = model.docvecs.most_similar([twain_real_to_vict])
sims

In [ ]:
london_male_to_female = model.docvecs[3] - model.docvecs[2] + model.docvecs[7]
sims = model.docvecs.most_similar([london_male_to_female])
sims

In [ ]:
london_nat_to_goth = model.docvecs[3] - model.docvecs[4] + model.docvecs[11]
sims = model.docvecs.most_similar([london_nat_to_goth])
sims

In [ ]:
hawthorne_to_london = model.docvecs[] - model.docvecs[4] + model.docvecs[11]